# Notebook Imports

In [1]:
import pandas as pd
import numpy as np

# Constants

In [2]:
TRAINING_DATA_FILE = "SpamData/02_Training/train-data.txt"
TEST_DATA_FILE = "SpamData/02_Training/test-data.txt"

TOKEN_SPAM_PROB_FILE = "SpamData/03_Testing/prob_spam.txt"
TOKEN_HAM_PROB_FILE = "SpamData/03_Testing/prob_nonspam.txt"
TOKEN_ALL_PROB_FILE = "SpamData/03_Testing/prob_all_token.txt"

TEST_FEATURE_MATRIX = "SpamData/03_Testing/test-features.txt"
TEST_TARGET_FILE = "SpamData/03_Testing/test-target.txt"

VOCAB_SIZE = 2500

# Read and Load features from .txt file into NumPy array

In [3]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype=int)

In [4]:
sparse_train_data

array([[   0,    2,    1,    1],
       [   0,    3,    1,    2],
       [   0,    4,    1,    1],
       ...,
       [5795, 2266,    0,    1],
       [5795, 2332,    0,    1],
       [5795, 2360,    0,    1]])

In [5]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype=int)

In [6]:
sparse_test_data

array([[   8,    2,    1,    1],
       [   8,    3,    1,    4],
       [   8,    4,    1,    2],
       ...,
       [5793, 2383,    0,    4],
       [5793, 2386,    0,    3],
       [5793, 2481,    0,    1]])

In [7]:
print('Nr of rows in training dataset:', sparse_train_data.shape[0])
print('Nr of rows in testing dataset:', sparse_test_data.shape[0])

Nr of rows in training dataset: 258351
Nr of rows in testing dataset: 117770


In [8]:
print('Nr of emails in training set', len(np.unique(sparse_train_data[:,0])))

Nr of emails in training set 4013


In [9]:
print('Nr of emails in testing set', len(np.unique(sparse_test_data[:,0])))

Nr of emails in testing set 1724


## How to create an empty DataFrame

In [10]:
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))

In [11]:
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [12]:
len(column_names)

2502

In [13]:
index_names = np.unique(sparse_train_data[ : , 0])
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [14]:
full_train_data = pd.DataFrame(columns=column_names, index=index_names)
full_train_data = full_train_data.fillna(value=0)
full_train_data

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5790,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5794,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Create Full Matrix from Sparse Matrix

In [15]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    """
    Form a full matrix from Sparse Matrix. Return Pandas DataFrame
    Keyword arguments: 
        sparse matrix --- numpy array
        nr_words --- size of words, Total number of tokens
        doc_idx --- position of the document id in sparse matrix. Default = 1st column
        word_idx --- position of the word id in sparse matrix. Default = 2nd column
        cat_idx --- position of the category(spam or non-spam). Default = 3d column
        freq_idx --- psition of the frequency of the word. Default = 4th column
    """
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:,0])
    full_matrix = pd.DataFrame(columns=column_names, index=doc_id_names)
    full_matrix.fillna(value=0, inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        label = sparse_matrix[i][cat_idx]
        word_id = sparse_matrix[i][word_idx]
        occurance = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurance
    
    full_matrix.set_index('DOC_ID', inplace=True)
    return full_matrix
    

In [16]:
%%time
make_full_matrix(sparse_train_data, VOCAB_SIZE).head()

Wall time: 8.64 s


,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,1,2,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6,0,0,2,4,0,3,14,0,...,0,0,0,0,0,0,0,0,0,0
4,1,5,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
%%time
full_training_matrix = make_full_matrix(sparse_train_data, VOCAB_SIZE)
full_training_matrix

Wall time: 8.94 s


,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,1,2,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6,0,0,2,4,0,3,14,0,...,0,0,0,0,0,0,0,0,0,0
4,1,5,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,0,3,1,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5790,0,1,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,0,3,1,0,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


# Training te Naive Bayes Model

## Calculating the probability of Spam

In [18]:
full_training_matrix.CATEGORY.sum()

1248

In [19]:
prob_spam = full_training_matrix.CATEGORY.sum()/len(full_training_matrix['CATEGORY'])
print('Probability of spam is:', prob_spam)

Probability of spam is: 0.310989284824321


## Total number of Words/Tokens

In [20]:
full_train_features = full_training_matrix.loc[:, full_training_matrix.columns != 'CATEGORY']
full_train_features

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,3,1,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5790,1,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,3,1,0,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
email_lengths = full_train_features.sum(axis=1)
email_lengths.head()

DOC_ID
0     87
1     53
2     40
3    183
4     43
dtype: int64

In [22]:
total_wc = email_lengths.sum()
total_wc

429252

## Nr of tokens in Spam and Ham emails

In [23]:
spam_lengths = email_lengths[full_training_matrix['CATEGORY'] == 1]
spam_wc = spam_lengths.sum()
spam_wc

176314

In [24]:
ham_lengths = email_lengths[full_training_matrix['CATEGORY'] == 0]
ham_wc = ham_lengths.sum()
ham_wc

252938

In [25]:
len(spam_lengths)

1248

In [26]:
len(ham_lengths)

2765

In [27]:
print('average words per email spam', round(spam_wc/len(spam_lengths)))
print('average words per email non-spam', round(ham_wc/len(ham_lengths)))

average words per email spam 141
average words per email non-spam 91


In [28]:
train_spam_tokens = full_train_features.loc[full_training_matrix.CATEGORY == 1]
train_spam_tokens

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885,1,0,0,2,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1887,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1889,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0


In [29]:
summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1
summed_spam_tokens.shape

(2500,)

In [30]:
summed_spam_tokens.head()

0    2179
1     935
2    1217
3    2022
4    1219
dtype: int64

## Summing the tokens occuring in ham

In [31]:
train_ham_tokens = full_train_features.loc[full_training_matrix.CATEGORY == 0]
train_ham_tokens

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1896,1,2,1,0,1,1,2,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1898,2,1,0,1,1,0,2,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1899,2,1,1,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1900,3,4,0,1,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1901,2,2,0,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,3,1,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5790,1,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,3,1,0,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
# Adding 1 to all the tokens due to Laplace Smoothing

summed_ham_tokens = train_ham_tokens.sum(axis=0) + 1
summed_ham_tokens.shape

(2500,)

In [33]:
summed_ham_tokens.head()

0    5484
1    2590
2    2045
3     938
4    1612
dtype: int64

In [34]:
summed_ham_tokens.tail()

2495     7
2496    25
2497    36
2498    23
2499    33
dtype: int64

In [35]:
train_ham_tokens[2499].sum() + 1

33

## P(Token| Spam) - Probablity that a Token occurs given the Email is Spam

In [36]:
# Adding 2500 to spam_wc due Laplace Smoothing at summed_spam_tokens

prob_spam_tokens = summed_spam_tokens/ (spam_wc + 2500)

In [37]:
prob_spam_tokens

0       0.012186
1       0.005229
2       0.006806
3       0.011308
4       0.006817
          ...   
2495    0.000123
2496    0.000034
2497    0.000022
2498    0.000034
2499    0.000006
Length: 2500, dtype: float64

In [38]:
prob_spam_tokens.sum()

0.9999999999999999

## P(Token| Ham) - Probablity that a Token occurs given the Email is Non-Spam

In [39]:
prob_ham_tokens = summed_ham_tokens/ (ham_wc + 2500)

In [40]:
prob_ham_tokens

0       0.021469
1       0.010139
2       0.008006
3       0.003672
4       0.006311
          ...   
2495    0.000027
2496    0.000098
2497    0.000141
2498    0.000090
2499    0.000129
Length: 2500, dtype: float64

In [41]:
prob_ham_tokens.sum()

1.0

## P(Token) - Probaility that a Token Occurs

In [42]:
prob_tokens_all = full_train_features.sum(axis=0)/total_wc

In [43]:
prob_tokens_all

0       0.017847
1       0.008207
2       0.007595
3       0.006891
4       0.006591
          ...   
2495    0.000063
2496    0.000068
2497    0.000089
2498    0.000063
2499    0.000075
Length: 2500, dtype: float64

In [44]:
prob_tokens_all.sum()

1.0

In [45]:
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_spam_tokens)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_ham_tokens)

## Prepare Test Data

In [46]:
full_test_data = pd.DataFrame(columns=column_names, index=index_names)
full_test_data = full_train_data.fillna(value=0)
full_test_data

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5790,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5794,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
sparse_test_data.shape

(117770, 4)

In [48]:
%%time
full_test_data = make_full_matrix(sparse_test_data, nr_words=VOCAB_SIZE)

Wall time: 4.13 s


In [49]:
full_test_data

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
8,1,0,0,1,4,2,1,2,4,1,...,0,0,0,0,0,0,0,0,0,0
12,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
14,1,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,1,0,2,1,1,2,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
17,1,0,0,0,0,1,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5783,0,2,1,0,2,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
5786,0,5,5,2,2,1,2,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5788,0,0,4,0,2,4,3,3,1,4,...,0,0,0,0,0,0,0,0,0,0


In [53]:
X_test = full_test_data.loc[:,full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY

In [54]:
np.savetxt(TEST_TARGET_FILE, y_test)
np.savetxt(TEST_FEATURE_MATRIX, X_test)

In [55]:
X_test.shape

(1724, 2500)